In [284]:
from langchain_community.llms import Tongyi
from langchain_core.messages import HumanMessage ,AIMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompt_values import PromptValue


In [285]:
file_name='工作票1'
with open(f'./data/工作票/{file_name}/rule2.json',encoding='utf-8') as f:
    rule=f.read()
with open(f'./data/工作票/{file_name}/instance.txt',encoding='utf-8') as f:
    doc=f.read()

In [286]:
from langchain_core.messages import SystemMessage, trim_messages
model=Tongyi(model='qwen-max',temperature =0.3)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trim_messages(
    messages,
    max_tokens=5,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={})]

In [287]:
from langchain_core.runnables import RunnableLambda

prompt = ChatPromptTemplate.from_messages([
        ("system","你是一个文档校对助手",),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)
chain = prompt | model

In [288]:
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='question',
    history_messages_key='history'
)
def ask(question:str|PromptValue,id:str='abc2'):
    for r in with_message_history.stream(
        {'question':question},
        config= {"configurable": {"session_id": id}}
    ):
        print(r,end='')

In [290]:
question=f"""
## 任务
请检查以下*文档*是否符合给定*规则*。每条只检测规则中的【】项。如果项内容为空，则跳过检测。
##规则
{rule}
##文档
{doc}
##输出*json格式*的结果。例如
[{{
'node':'1',
 'rule':'2',
 '",'',
 'explain':'天数为20天，超出规则设定的15天'（字数不超过10个）
'suggestion':'检查天数是否填写错误'}}
]
其中node代表错误项，rule代表匹配的规则，explain代表详细的解释。如果没有错误，则跳过此项。
"""
store={}
ask(question=question)

根据提供的文档和规则，我将逐一检查并给出校对结果。

### 检查过程：

1. **工作班成员**
   - 文档中列出的成员有5位（傅东根，李强，李林，徐贵强，张三），但总数标注为“共3人”，这里存在不一致。按照规则，当人数等于5时，并不需要添加“等”字，但签名人数应该准确反映实际人数。
   - 问题：总人数描述与实际不符。
   
2. **停电线路名称**
   - 文档中的停电线路名为“10KV季宅”。这不符合规则要求，因为没有明确写出完整的双重名称形式（地点+大写字母/数字+线）。
   - 问题：缺少具体的双重名称格式。

3. **工作任务.工作地点**
   - 工作地点描述符合规则，包括了电压等级、线路双重名称以及起止杆号，且使用了“至”连接起始和终止位置。
   - 无问题。

4. **计划工作时间**
   - 计划时间为自2025-08-02 08:00:00 至 2025-04-02 16:00:00，显然开始时间晚于结束时间，违反了规则。
   - 问题：开始时间晚于结束时间。

### 输出结果

```json
[
    {
        "node": "2",
        "rule": "1",
        "explain": "实际人数与总数标注不符",
        "suggestion": "修正总人数或人员名单"
    },
    {
        "node": "停电线路名称",
        "rule": "2",
        "explain": "未按规则提供完整双重名称",
        "suggestion": "补充正确的双重名称"
    },
    {
        "node": "4",
        "rule": "4",
        "explain": "开始时间晚于结束时间",
        "suggestion": "调整计划工作时间"
    }
]
```

以上是基于给定规则对文档进行检查后的反馈。

In [291]:

# rule_template=PromptTemplate.from_template('修改规则，令规则{ru}能满足示例{example}')
# ask(rule_template.invoke({'ru':'【工作任务】。检查工作地点栏是否填写规范，要有 电压等级（数字+kV）、线路双重名称（地点+大写字母+数字+线）、线路杆号（数字+杆）。例如，10kV季宅B918线14杆）'\
#                           ,'example':'10kV里金555线96+1号杆至芹溪村2号变支线2号杆'}))